# 1. Authenticate & Import

In [ ]:
import ee
# 网不好的时候自动重试ee.Initialize()
print("Try init ee...", end=" ")
eeInited, tryCounts = False, 0
while not eeInited:
    print("%d" % tryCounts, end=" ")
    try:
        ee.Initialize()
    except Exception as e:
        if (str(e)[0:23]=="Please authorize access"):
            ee.Authenticate()
        tryCounts += 1
    else:
        eeInited = True
        print("\nee initialized!")

import geemap

In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import gc
import time
import random
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import cv2

import gdal
import utils
import common
import dataset
import GetMapTiles
from models.ResNet import ResNet101

# 2. Import Earth Engine Data

In [ ]:
# Boundary and Grid-----------------------------------------------------------------------------------------------------
worldBoundary = ee.FeatureCollection("users/liuph/shape/WorldBoundary")
ChinaBoundary = ee.FeatureCollection("users/410093033/China")
WorldGrid5d = ee.FeatureCollection("users/liuph/shape/WorldGrid5dC5")

# Fusion of landcover products------------------------------------------------------------------------------------------
# forest_fuse = forest_gong.add(forest_ygs).add(forest_liu).add(forest_chen).add(forest_hs).rename("ForestFuse")
# forest_fuse = ee.ImageCollection('users/sysushiqian/forestFuse2020').min().rename("forest_fuse")
forest_fuse = ee.ImageCollection('users/sysushiqian/forestFuse2020_new').min().rename("ForestFuse")

In [ ]:
totalGrid = ee.FeatureCollection("users/sunly3456/globalGrids/globalGt60Grids/forest23gt60gridsAll")
totalGridIdxList = list(range(1, 100))

dataPath = r"E:/SZT/Data/Forest/densePredData"
predPath = r"./densePred"


# 按totalGridIdxList中的索引，依次预测对应0.1度格网
for gridIdx in totalGridIdxList:
    ID_5d = totalGrid.filterMetadata("orderID", "equals", gridIdx).first().getNumber("gridID").getInfo()
    fNamePostfix = "_%d_%d.tif" % (ID_5d, gridIdx)

    curFFPath = dataPath + "/FF01" + fNamePostfix
    curGEPath = dataPath + "/GE17" + fNamePostfix
    predSavePath = predPath + "/densePred" + fNamePostfix

    print("\n[Grid5d=%d Grid01d=%d] %s" % (ID_5d, gridIdx, time.ctime()))
    print("%s" % "-"*80)

    if os.path.exists(predSavePath):
        print("grid01d %d prediction found" % gridIdx)

        time.sleep(2)

        geImg = cv2.imread(curGEPath, -1)[:, :, [2, 1, 0]]
        ffImg = cv2.imread(curFFPath, -1)
        predImg = cv2.imread(predSavePath, -1)

        # 预览
        plt.figure(figsize=(60, 20))
        plt.subplot(1, 3, 1)
        plt.imshow(geImg)
        plt.title("Google Earth")
        plt.subplot(1, 3, 2)
        ffImg[0, 0] = 6
        plt.imshow(ffImg, cmap=mpl.colors.ListedColormap(["#FF0000", "#FF0000", "#0000FF", "#0000FF", "#00FF00", "#00FF00"], "forestfusion", N=6))
        plt.title("forest products fusion")
        plt.subplot(1, 3, 3)
        predImg[0, 0] = 5
        plt.imshow(predImg, cmap=mpl.colors.ListedColormap(["#00FF00", "#555555", "#FF0000", "#FF0000", "#0000FF"], "pred", N=5))
        plt.title("dense prediction")
        plt.show()